In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains

#*******************************************************************************************************************
import sys
import requests
sys.path.insert(0, os.path.dirname(__file__).replace('parsing-new-script','global-files/'))
from GlobalVariable import *
from GlobalFunctions import *
#*******************************************************************************************************************

import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
url='https://www.canadianinstitute.com/conferences/?type=c5_event'

In [4]:
driver.get(url)
time.sleep(5)

In [5]:
while True:
    try:
        driver.find_element_by_css_selector('div[class=list-more-items-btn]').click()
        time.sleep(2)
    except:
        break

In [6]:
soup = bs(driver.page_source,'lxml')
cards=soup.find_all('div',{'class':'event-block'})
len(cards)

486

In [7]:
cain=[]
sag=1
for z in cards:
    tit=' '.join(z.find('h4').text.split())
    link=z.find('a')['href']
    tid=' '.join(z.find('div',{'class':'block-meta event-date-location'}).text.split())
    onw=' '.join(z.find_all('a')[-1].text.split())
    driver.get(link)
    time.sleep(2)
    soupn = bs(driver.page_source,'lxml')
    typ=soupn.find('div',{'class':'event-meta-container'}).find('a').text
    if 'Person' in typ:
        on_off='0'
    else:
        on_off='1'
    try:
        pre=soupn.find('h4',{'class':'event-title-prefix'}).text
    except:
        pre=''
    ti=soupn.find('h1',{'class':'event-title'}).text
    title=(pre+' '+ti).strip()
    dat=' '.join(soupn.find('span',{'class':'event-date'}).text.split())
    try:
        loc=' '.join(soupn.find('span',{'class':'event-location'}).text.split())
    except:
        loc=''
#     try:
#         info=soupn.find('div',{'class':'content-block body-content container'}).find('h4').text
#     except:
#         info=soupn.find('div',{'class':'content-block body-content container'}).find('h3').text
#     try:
#         pa=soupn.find('div',{'class':'content-block body-content container'}).find('p').text
#     except:
#         pa=''
#     event_info=info+pa    
    event_info=' '.join(soupn.find('div',{'class':'header-cta-container'}).find('p').get_text(separator='.').split())
    xam=soupn.find_all('div',{'class':'speaker'})
    sps=[]
    for e in xam:
        nam=e.find('h4').get_text(separator=' ')
        titll=e.find('p').get_text(separator=' ')
        ln=''
        grou=[nam,titll,ln]
        sps.append(grou)
    speak=[]
    for f in sps:
        ts={'name':f[0],
            'title':f[1],
            'link':f[2]
           }
        speak.append(ts)
    if speak==[] or speak=='':
        Speakerlist=''
    else:
        Speakerlist=json.dumps(speak)
        
     #from HERE*****
    dat=dat.replace('–','-')
    if '(' in dat:
        dat=dat.split('(')[0].strip()
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat
        #try:
#             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
#             mad=ma.group()

        if '-' in mad:
            st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        elif 'and'in mad:
            st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        elif '&' in mad:
            st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

        else:
            st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        #print(date__,st,en)
        ####################
        if any(c.isalpha() for c in st)==False:
        #      print('leg')
            pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
            sapa=pa.group()
            start=st+' '+sapa
            end=en
        #                        02        April      2022
        elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
         #   print('awujale')
            start=st
            end=en

        #*                        April       02      2022
        elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
        #       print('ala')
            pa=re.search('(\d{4})',st).group()
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()
            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',en).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba     

        #                      02     2022
        elif re.search('\d{1,2}\s+\d{4}',en):
        #      print('kum')
            pa=re.search('(\d{4})',en).group()#, maxsplit=0)
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()

            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',st).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba

        #                      02         April
        elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
        #        print('is')
            pa=re.search('(\d{4})',en)#, maxsplit=0)
            sapa=pa.group()
            start=st+' '+sapa
            end=en

        #*                      April           02
        elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
        #      print('bad')
            pa=re.search('(\d{4})',en).group()#, maxsplit=0)
            sa=re.search('([A-Sa-z]{3,9})',st).group()
            ta=re.search('(\d{1,2})',st).group()

            start=ta+' '+sa+' '+pa
            #
            ba=re.search('(\d{4})',en).group()
            ca=re.search('([A-Sa-z]{3,9})',en).group()
            da=re.search('(\d{1,2})',en).group()
            end=da+' '+ca+' '+ba

        else:
        #     print('shik')
            start=end=''
        if start=='':
            start_date=end_date=''
        else:
            pick=[start, end]
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
            date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
            start_date=date_[0]
            end_date=date_[1]
        
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            sha=location.split(',')
            try:
                gu(sha[-3])
            except:
                try:
                    gu(sha[-2])
                except:
                    try:
                        gu(sha[-1])
                    except Exception as e:
                        print(location, "; url didn't go through")
                        print(e)
                        return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")
        
        
    if on_off=='1' or loc==' ' or loc=='':
        city=''
        country=''
        venue=''
        googlePlaceUrl=''
    else:
        city=loc
        country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)
    ticket_list=''
    org_name='The Canadian Institute'
    org_web='https://www.canadianinstitute.com/'
    org_pro='We conceive, develop and host authoritative conferences, forums and summits across multiple sectors. Our goal at C5 Group is single-minded: to drive business and professional success. To achieve this, we offer a range of targeted tools and products. Through our various services, we provide knowledge, intelligence, insight, analysis, connections and networking, all designed to create advantage for our customers.'
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    category=''
    Speakerlist='' 
    time_=''
    event_web=link
    mail_=json.dumps(['customerservice@canadianinstitute.com'])
    cain.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsor,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])
    sag+=1
    for d in range(0,50*100,20):
        if sag==d:
            print(d,' cards done')
print('Done')

20  cards done
40  cards done
 ;country didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":".Lx2b0d"}
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A5B8F3+2406643]
	Ordinal0 [0x009EAF31+1945393]
	Ordinal0 [0x008DC748+837448]
	Ordinal0 [0x009092E0+1020640]
	Ordinal0 [0x0090957B+1021307]
	Ordinal0 [0x00936372+1205106]
	Ordinal0 [0x009242C4+1131204]
	Ordinal0 [0x00934682+1197698]
	Ordinal0 [0x00924096+1130646]
	Ordinal0 [0x008FE636+976438]
	Ordinal0 [0x008FF546+980294]
	GetHandleVerifier [0x00CC9612+2498066]
	GetHandleVerifier [0x00CBC920+2445600]
	GetHandleVerifier [0x00AF4F2A+579370]
	GetHandleVerifier [0x00AF3D36+574774]
	Ordinal0 [0x009F1C0B+1973259]
	Ordinal0 [0x009F6688+1992328]
	Ordinal0 [0x009F6775+1992565]
	Ordinal0 [0x009FF8D1+2029777]
	BaseThreadInitThunk [0x75FFFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77907A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77907A4E+238]

 ; url didn't go th

Vancouver, BC ; url didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":".Lx2b0d"}
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A5B8F3+2406643]
	Ordinal0 [0x009EAF31+1945393]
	Ordinal0 [0x008DC748+837448]
	Ordinal0 [0x009092E0+1020640]
	Ordinal0 [0x0090957B+1021307]
	Ordinal0 [0x00936372+1205106]
	Ordinal0 [0x009242C4+1131204]
	Ordinal0 [0x00934682+1197698]
	Ordinal0 [0x00924096+1130646]
	Ordinal0 [0x008FE636+976438]
	Ordinal0 [0x008FF546+980294]
	GetHandleVerifier [0x00CC9612+2498066]
	GetHandleVerifier [0x00CBC920+2445600]
	GetHandleVerifier [0x00AF4F2A+579370]
	GetHandleVerifier [0x00AF3D36+574774]
	Ordinal0 [0x009F1C0B+1973259]
	Ordinal0 [0x009F6688+1992328]
	Ordinal0 [0x009F6775+1992565]
	Ordinal0 [0x009FF8D1+2029777]
	BaseThreadInitThunk [0x75FFFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77907A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77907A4E+238]

100  cards done
120  cards done
140  ca

KeyboardInterrupt: 

In [8]:
len(cain)

179

In [9]:
cain

[['https://www.canadianinstitute.com/women-leaders-ip-law',
  'Women Leaders in IP Law',
  '2022-05-24',
  '2022-05-25',
  '',
  'Engage with women leaders in intellectual property law who will cover all facets of IP – patents, copyrights, trademarks and more.',
  '',
  'We conceive, develop and host authoritative conferences, forums and summits across multiple sectors. Our goal at C5 Group is single-minded: to drive business and professional success. To achieve this, we offer a range of targeted tools and products. Through our various services, we provide knowledge, intelligence, insight, analysis, connections and networking, all designed to create advantage for our customers.',
  'The Canadian Institute',
  'https://www.canadianinstitute.com/',
  '',
  '',
  '',
  '',
  '',
  'Westin Washington, D.C. City Center, Washington, DC',
  '',
  'Westin Washington, D.C. City Center, Washington, DC',
  'https://www.canadianinstitute.com/women-leaders-ip-law',
  'https://www.google.com/maps/pl

In [10]:
cain_dict=cain


cain_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=cain_dict)
cain_df.to_csv("cain.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# truce=[]
# for k in range(1,25):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(1)
#     soupn=bs(driver.page_source,'lxml')
#     dat=' '.join(soupn.find('span',{'class':'event-date'}).text.split())
#     try:
#         loc=' '.join(soupn.find('span',{'class':'event-location'}).text.split())
#     except:
#         loc=''
#     typ=soupn.find('div',{'class':'event-meta-container'}).find('a').text
#     if 'Person' in typ:
#         on_off='0'
#     else:
#         on_off='1'
#     #from HERE*****
#     dat=dat.replace('–','-')
#     if '(' in dat:
#         dat=dat.split('(')[0].strip()
#     if dat=='':
#         start_date=end_date=''
#     else:
#         mad=dat
#         #try:
# #             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
# #             mad=ma.group()

#         if '-' in mad:
#             st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#         elif 'and'in mad:
#             st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#         elif '&' in mad:
#             st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

#         else:
#             st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#             en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
#         #print(date__,st,en)
#         ####################
#         if any(c.isalpha() for c in st)==False:
#         #      print('leg')
#             pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
#             sapa=pa.group()
#             start=st+' '+sapa
#             end=en
#         #                        02        April      2022
#         elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
#          #   print('awujale')
#             start=st
#             end=en

#         #*                        April       02      2022
#         elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
#         #       print('ala')
#             pa=re.search('(\d{4})',st).group()
#             sa=re.search('([A-Sa-z]{3,9})',st).group()
#             ta=re.search('(\d{1,2})',st).group()
#             start=ta+' '+sa+' '+pa
#             #
#             ba=re.search('(\d{4})',en).group()
#             ca=re.search('([A-Sa-z]{3,9})',en).group()
#             da=re.search('(\d{1,2})',en).group()
#             end=da+' '+ca+' '+ba     

#         #                      02     2022
#         elif re.search('\d{1,2}\s+\d{4}',en):
#         #      print('kum')
#             pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#             sa=re.search('([A-Sa-z]{3,9})',st).group()
#             ta=re.search('(\d{1,2})',st).group()

#             start=ta+' '+sa+' '+pa
#             #
#             ba=re.search('(\d{4})',en).group()
#             ca=re.search('([A-Sa-z]{3,9})',st).group()
#             da=re.search('(\d{1,2})',en).group()
#             end=da+' '+ca+' '+ba

#         #                      02         April
#         elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
#         #        print('is')
#             pa=re.search('(\d{4})',en)#, maxsplit=0)
#             sapa=pa.group()
#             start=st+' '+sapa
#             end=en

#         #*                      April           02
#         elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
#         #      print('bad')
#             pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#             sa=re.search('([A-Sa-z]{3,9})',st).group()
#             ta=re.search('(\d{1,2})',st).group()

#             start=ta+' '+sa+' '+pa
#             #
#             ba=re.search('(\d{4})',en).group()
#             ca=re.search('([A-Sa-z]{3,9})',en).group()
#             da=re.search('(\d{1,2})',en).group()
#             end=da+' '+ca+' '+ba

#         else:
#         #     print('shik')
#             start=end=''
#         if start=='':
#             start_date=end_date=''
#         else:
#             pick=[start, end]
#             spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
#             date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
#             start_date=date_[0]
#             end_date=date_[1]
# #     try:
# #         try:
# #             info=' '.join(soupn.find('div',{'class':'content-block body-content container'}).find('h4').text.split())
# #         except:
# #             info=' '.join(soupn.find('div',{'class':'content-block body-content container'}).find('h3').text.split())
# #         try:
# #             pa=' '.join(soupn.find('div',{'class':'content-block body-content container'}).find('p').text.split())
# #         except:
# #             pa=''
# #         event_info=info+pa    
# #     except:
#     event_info=' '.join(soupn.find('div',{'class':'header-cta-container'}).find('p').get_text(separator='.').split())
        
#     xam=soupn.find_all('div',{'class':'speaker'})
#     sps=[]
#     for e in xam:
#         nam=e.find('h4').get_text(separator=' ')
#         titll=e.find('p').get_text(separator=' ')
#         ln=''
#         grou=[nam,titll,ln]
#         sps.append(grou)
#     speak=[]
#     for f in sps:
#         ts={'name':f[0],
#             'title':f[1],
#             'link':f[2]
#            }
#         speak.append(ts)
#     if speak==[] or speak=='':
#         Speakerlist=''
#     else:
#         Speakerlist=json.dumps(speak)
#     truce.append([typ,on_off,start_date,end_date,event_info,Speakerlist])
# truce